In [114]:
import urllib.request
import bs4
import pandas

response = urllib.request.urlopen('https://vk.com/topic-22522055_25921627?offset=20')
soup = bs4.BeautifulSoup(response, 'html.parser')

In [122]:
nodes = soup.findAll('div', {'class': 'post_item'})

20


In [137]:
import datetime as _dt
import re as _re

monthRus = [
    'янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'
]

vkDateRegex = _re.compile('([0-9]{1,2})\W([а-я]{3})\W([0-9]{4})\Wв\W([0-9]{1,2}):([0-9]{1,2})', _re.IGNORECASE)

In [132]:
for node in nodes:
    fromId = node.find('a', {'class': 'pi_author'}).get('href')[1:]
    toId = node.find('a', {'class': 'mem_link'})
    
    if (toId):
        toId = toId.get('href')[1:]
    
    date = node.find('a', {'class', 'item_date'})
    
    likes = node.find('b', {'class', 'v_like'})
    likes = likes.text if likes else 0
    

In [165]:
# '1 фев 2012 в 11:48' ([0-9]{1,2})\W([а-я]{3})\W([0-9]{4})\Wв\W([0-9]{1,2}):([0-9]{1,2})

day, month, year, hours, minutes = vkDateRegex.findall('1 фев 2012 в 11:48')[0]
date = _dt.datetime(int(year), monthRus.index(month)+1, int(day), int(hours), int(minutes), 0)
date

datetime.datetime(2012, 2, 1, 11, 48)

In [154]:
# import time    
# time.strftime('%Y-%m-%d %H:%M:%S')
# datetime.strptime( "2009-11-12 23:18:53", "%Y-%m-%d %H:%M:%S" )
monthRus.index('янв')

0